In [1]:
#get processed data

import pandas as pd

lut = pd.read_csv("https://github.com/Chike-Ode/Imputation_Research_Project/blob/master/data/processed/lutein.csv?raw=true")

sug = pd.read_csv("https://github.com/Chike-Ode/Imputation_Research_Project/blob/master/data/processed/sugars.csv?raw=true")

wat=pd.read_csv("https://github.com/Chike-Ode/Imputation_Research_Project/blob/master/data/processed/water.csv?raw=true")

#dropping Name column
lut=lut.drop('name',axis=1)
sug=sug.drop('name',axis=1)
wat=wat.drop('name',axis=1)


In [3]:
pip install MissForestExtra

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install missingpy

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install scikit-learn==0.22.1

  Running setup.py clean for scikit-learn
Failed to build scikit-learn
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2
    Running setup.py install for scikit-learn: started
    Running setup.py install for scikit-learn: finished with status 'error'
  Rolling back uninstall of scikit-learn
  Moving to c:\users\gabri\anaconda3\lib\site-packages\scikit_learn-0.24.2.dist-info\
   from C:\Users\gabri\anaconda3\Lib\site-packages\~cikit_learn-0.24.2.dist-info
  Moving to c:\users\gabri\anaconda3\lib\site-packages\sklearn\
   from C:\Users\gabri\anaconda3\Lib\site-packages\~klearn
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\gabri\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\gabri\\AppData\\Local\\Temp\\pip-install-8591uz45\\scikit-learn_34882d4bceb7493ab8dc61a9564241f3\\setup.py'"'"'; __file__='"'"'C:\\Users\\gabri\\AppData\\Local\\Temp\\pip-install-8591uz45\\scikit-learn_34882d4bceb7493ab8dc61a9564241f3\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\gabri\AppData\Local\Temp\pip-wheel-exxdqgex'
       cwd: C:\Users\gabri\AppData\Local\Temp\pip-install-8591uz45\scikit-learn_34882d4bceb7493ab8dc61a9564241f3\
  Complete output (35 lines):
  Partial import of sklearn during the build process.
  No module named 'numpy.distutils._msvccompi

In [13]:
#replace NaN using MICE algorithm
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

from missingpy import MissForest

# Make an instance and perform the imputation
imp = MissForest()

#lutein mcar
#Need to drop lutein mar and true lutein columns
lutmcar=lut.copy()
lutmcar=lutmcar.drop(['lutein_zeaxanthin_mar','lutein_zeaxanthin'],axis=1)
imp.fit(lutmcar)
lutmcar2=imp.transform(lutmcar)

lutmcarimp = pd.DataFrame(lutmcar2, columns = lutmcar.columns)

#lutein mar
#Need to drop lutein mcar and true lutein columns
lutmar=lut.copy()
lutmar=lutmar.drop(['lutein_zeaxanthin_mcar','lutein_zeaxanthin'],axis=1)
imp.fit(lutmar)
lutmar2=imp.transform(lutmar)

lutmarimp = pd.DataFrame(lutmar2, columns = lutmar.columns)

#sugars mcar
#Need to drop sugar mar and true sugar columns
sugmcar=sug.copy()
sugmcar=sugmcar.drop(['sugars_mar','sugars'],axis=1)
imp.fit(sugmcar)
sugmcar2=imp.transform(sugmcar)
sugmcarimp = pd.DataFrame(sugmcar2, columns = sugmcar.columns)


#sugars mar
#Need to drop sugar mcar and true sugar columns
sugmar=sug.copy()
sugmar=sugmar.drop(['sugars_mcar','sugars'],axis=1)
imp.fit(sugmar)
sugmar2=imp.transform(sugmar)
sugmarimp = pd.DataFrame(sugmar2, columns = sugmar.columns)


#water mcar
#Need to drop water mar and true water columns
watmcar=wat.copy()
watmcar=watmcar.drop(['water_mar','water'],axis=1)
imp.fit(watmcar)
watmcar2=imp.transform(watmcar)
watmcarimp = pd.DataFrame(watmcar2, columns = watmcar.columns)


#water mar
#Need to drop water mcar and true water columns
watmar=wat.copy()
watmar=watmar.drop(['water_mcar','water'],axis=1)
imp.fit(watmar)
watmar2=imp.transform(watmar)
watmarimp = pd.DataFrame(watmar2, columns = watmar.columns)


Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3


In [14]:
#adding back the column with true values

lutmcarimp['true_lut'] = lut['lutein_zeaxanthin'].values
lutmarimp['true_lut'] = lut['lutein_zeaxanthin'].values

sugmcarimp['true_sug'] = sug['sugars'].values
sugmarimp['true_sug'] = sug['sugars'].values

watmcarimp['true_wat'] = wat['water'].values
watmarimp['true_wat'] = wat['water'].values

In [15]:
#calculating rmse for each columns and missing type
import math
import numpy as np


meth='RF'
results=[]

#lutein mcar
dat='lutein'
dt=lutmcarimp

dt['sqrt']=(dt['true_lut']-dt['lutein_zeaxanthin_mcar'])**2
lutmcarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",lutmcarrmse)
results.append([dat,'mcar',meth,lutmcarrmse])

#lutein mar
dt=lutmarimp

dt['sqrt']=(dt['true_lut']-dt['lutein_zeaxanthin_mar'])**2
lutmarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",lutmarrmse)
results.append([dat,'mar',meth,lutmarrmse])

#sugars mcar
dat='sugars'
dt=sugmcarimp

dt['sqrt']=(dt['true_sug']-dt['sugars_mcar'])**2
sugmcarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",sugmcarrmse)
results.append([dat,'mcar',meth,sugmcarrmse])

#sugars mar
dt=sugmarimp

dt['sqrt']=(dt['true_sug']-dt['sugars_mar'])**2
sugmarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",sugmarrmse)
results.append([dat,'mar',meth,sugmarrmse])

#water mcar
dat='water'
dt=watmcarimp

dt['sqrt']=(dt['true_wat']-dt['water_mcar'])**2
watmcarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",watmcarrmse)
results.append([dat,'mcar',meth,watmcarrmse])

#water mcar
dt=watmarimp

dt['sqrt']=(dt['true_wat']-dt['water_mar'])**2
watmarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",watmarrmse)
results.append([dat,'mar',meth,watmarrmse])

print(results)

RMSE: 377.0558604550902
RMSE: 382.2273290339469
RMSE: 3.5160120184776096
RMSE: 1.4863895382687973
RMSE: 1.5241258470635468
RMSE: 1.6890076573919701
[['lutein', 'mcar', 'RF', 377.0558604550902], ['lutein', 'mar', 'RF', 382.2273290339469], ['sugars', 'mcar', 'RF', 3.5160120184776096], ['sugars', 'mar', 'RF', 1.4863895382687973], ['water', 'mcar', 'RF', 1.5241258470635468], ['water', 'mar', 'RF', 1.6890076573919701]]


In [16]:
res = pd.DataFrame(results, columns = ['Column', 'Missing type','Imputation method','RMSE'])
print(res)

   Column Missing type Imputation method        RMSE
0  lutein         mcar                RF  377.055860
1  lutein          mar                RF  382.227329
2  sugars         mcar                RF    3.516012
3  sugars          mar                RF    1.486390
4   water         mcar                RF    1.524126
5   water          mar                RF    1.689008
